# [CSC8101] Big Data Analytics - 2022 Spark Coursework

## Coursework overview

### Inputs

- **NYC Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **NYC Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Periodically download your notebook to your computer as backup and safety measure against accidental file deletion.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [0]:
## global imports

try:
    from pyspark.sql.functions import *
    from pyspark.sql import SparkSession
    from pyspark.sql.types import *
    import pandas as pd
# Additional global imports
    import pyspark.sql as ps
    import pyspark.sql.functions as pf
    spark = SparkSession.builder\
   .getOrCreate()
    
except Exception as err:
    print(f'Import error{err}')


# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/NewcastleComputingScience/csc8101-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))

# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
    else:
        _trips_12 = _trips_11

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)

    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

In [0]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'XXL'
DATA_FORMAT = 'delta'

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

# uncomment line only for small datasets
# trips.take(1)

Trips dataset loaded!
 ---
 Size: XXL
 Format: delta
 Tables loaded: /FileStore/tables/taxi/taxi-XXL-delta/
 Number of trips (dataset rows): 132,396,785

In [0]:
print_count(trips)

Row count: 132,396,785

In [0]:
# dataset schemas
trips.printSchema()

root
-- index: long (nullable = true)
-- VendorID: double (nullable = true)
-- tpep_pickup_datetime: string (nullable = true)
-- tpep_dropoff_datetime: string (nullable = true)
-- passenger_count: double (nullable = true)
-- trip_distance: double (nullable = true)
-- RatecodeID: double (nullable = true)
-- store_and_fwd_flag: string (nullable = true)
-- PULocationID: long (nullable = true)
-- DOLocationID: long (nullable = true)
-- payment_type: double (nullable = true)
-- fare_amount: double (nullable = true)
-- extra: double (nullable = true)
-- mta_tax: double (nullable = true)
-- tip_amount: double (nullable = true)
-- tolls_amount: double (nullable = true)
-- improvement_surcharge: double (nullable = true)
-- total_amount: double (nullable = true)
-- congestion_surcharge: double (nullable = true)
-- cab_type: string (nullable = true)
-- lpep_pickup_datetime: string (nullable = true)
-- lpep_dropoff_datetime: string (nullable = true)
-- ehail_fee: double (nullable = true)
-- trip_type: double (nullable = true)

In [0]:
display(trips[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(5))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
151,239,1.5,1.0,9.95
239,246,2.6,1.0,16.3
236,236,0.0,3.0,5.8
193,193,0.0,5.0,7.55
193,193,0.0,5.0,55.55


In [0]:
zone_names.printSchema()

root
-- LocationID: long (nullable = true)
-- Borough: string (nullable = true)
-- Zone: string (nullable = true)
-- service_zone: string (nullable = true)

In [0]:
display(zone_names.take(5))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')]`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.438 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

In [0]:
####################################################################################################################
######################################## Solution implementation to task 1.1 #######################################
####################################################################################################################

### selecting necessary features
cols = ['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']
trips = trips.select(cols)

def t11_remove_zeros(df):
    remove_0_records = df.where('trip_distance != 0').where('passenger_count != 0')\
    .where('total_amount !=0')
 
    return remove_0_records

In [0]:
# execute task 1.1
trips_11 = t11_remove_zeros(trips)

print_count(trips_11)

## uncomment only for smaller datasets
display(trips_11.take(10))

Row count: 125,535,704

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
151,239,1.5,1.0,9.95
239,246,2.6,1.0,16.3
163,229,1.3,1.0,9.05
229,7,3.7,1.0,18.5
141,234,2.1,2.0,13.0
246,162,2.8,2.0,19.55
238,151,0.7,1.0,8.5
163,25,8.7,1.0,42.95
224,25,6.3,1.0,28.5
141,234,2.7,1.0,15.3


In [0]:
####################################################################################################################
######################################## Solution implementation to task 1.2 #######################################
####################################################################################################################

def t12_remove_outliers(df):
    
    MAD_distance = df.groupby('PULocationID').agg(expr('percentile(trip_distance, array(0.5))')[0].alias('median'))\
            .join(trips_11, "PULocationID", "left").withColumn("difference_median",
            abs(col('trip_distance')- col('median'))).groupby('PULocationID', 'median')\
            .agg(expr('percentile(difference_median, array(0.5))')[0].alias('median_absolute_difference'))\
            .withColumn('MAD',col('median_absolute_difference')*1.483)
    
    MAD_amount = df.groupby('PULocationID').agg(expr('percentile(total_amount, array(0.5))')[0].alias('amount_median'))\
            .join(trips_11, "PULocationID", "left").withColumn("amount_difference_median",
            abs(col('total_amount')- col('amount_median'))).groupby('PULocationID', 'amount_median')\
            .agg(expr('percentile(amount_difference_median, array(0.5))')[0].alias('amount_median_absolute_difference'))\
            .withColumn('MAD_amount',col('amount_median_absolute_difference')*1.483)

    outliersremoved_distance = df.join(MAD_distance, "PULocationID", "left")\
        .filter(abs(col("trip_distance")- col("median")) <= (col("MAD")*3.5)).select(cols)
    
    outliersremoved_amount = df.join(MAD_amount, "PULocationID", "left")\
        .filter(abs(col("total_amount")- col("amount_median")) <= (col("MAD_amount")*3.5)).select(cols)
    
    outliersremoved = outliersremoved_distance.join(outliersremoved_amount,
                                   ['PULocationID','DOLocationID','trip_distance','passenger_count','total_amount'],'inner').drop_duplicates()

    return outliersremoved

In [0]:
# execute task 1.2
try:
    trips_12 = t12_remove_outliers(trips_11)
    print_count(trips_12)
    display(trips_12.take(10))

except Exception as err:
    print(f'{err}')

Row count: 44,135,513

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
4,48,3.9,1.0,23.3
4,68,2.87,1.0,15.8
4,75,6.58,5.0,25.3
4,79,0.39,5.0,9.36
4,79,0.81,2.0,12.96
4,88,3.9,3.0,24.8
4,100,2.9,1.0,27.95
4,144,1.44,1.0,18.51
4,170,2.37,2.0,17.8
4,224,0.94,6.0,11.16


## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitabilty = total_amount / trip_distance`.

In [0]:
####################################################################################################################
######################################## Solution implementation to task 2.1 #######################################
####################################################################################################################

def t21_join_zones(df, zones_df = zone_names):
    # input: output of task 1.2 and zone_names dataset
    
    joined = df.join(zones_df.select('LocationID','Zone'),df.PULocationID == zones_df.LocationID,'inner')\
                .withColumnRenamed('Zone','PUzone').drop('LocationID')\
                .join(zones_df.select('LocationID','Zone'),df.DOLocationID == zones_df.LocationID,'inner')\
                .withColumnRenamed('Zone','DOzone')
    
    return joined

In [0]:
# execute task 2.1
trips_21 = t21_join_zones(trips_12, zones_df = zone_names)

print_count(trips_21)
display(trips_21.take(10))

Row count: 44,135,513

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount,PUzone,LocationID,DOzone
4,48,3.9,1.0,23.3,Alphabet City,48,Clinton East
4,68,2.87,1.0,15.8,Alphabet City,68,East Chelsea
4,75,6.58,5.0,25.3,Alphabet City,75,East Harlem South
4,79,0.39,5.0,9.36,Alphabet City,79,East Village
4,79,0.81,2.0,12.96,Alphabet City,79,East Village
4,88,3.9,3.0,24.8,Alphabet City,88,Financial District South
4,100,2.9,1.0,27.95,Alphabet City,100,Garment District
4,144,1.44,1.0,18.51,Alphabet City,144,Little Italy/NoLiTa
4,170,2.37,2.0,17.8,Alphabet City,170,Murray Hill
4,224,0.94,6.0,11.16,Alphabet City,224,Stuy Town/Peter Cooper Village


In [0]:
####################################################################################################################
######################################## Solution implementation to task 2.2 #######################################
####################################################################################################################

def t22_calc_profit(df):
    # input: output of task 2.1
    unitProf = trips_21.withColumn('unit_profitability',round(col('total_amount')/col('trip_distance'),2))
    return unitProf

In [0]:
# execute task 2.2
trips_22 = t22_calc_profit(trips_11)

print_count(trips_22)
display(trips_22.take(10))

Row count: 44,135,513

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount,PUzone,LocationID,DOzone,unit_profitability
4,48,3.9,1.0,23.3,Alphabet City,48,Clinton East,5.97
4,68,2.87,1.0,15.8,Alphabet City,68,East Chelsea,5.51
4,75,6.58,5.0,25.3,Alphabet City,75,East Harlem South,3.84
4,79,0.39,5.0,9.36,Alphabet City,79,East Village,24.0
4,79,0.81,2.0,12.96,Alphabet City,79,East Village,16.0
4,88,3.9,3.0,24.8,Alphabet City,88,Financial District South,6.36
4,100,2.9,1.0,27.95,Alphabet City,100,Garment District,9.64
4,144,1.44,1.0,18.51,Alphabet City,144,Little Italy/NoLiTa,12.85
4,170,2.37,2.0,17.8,Alphabet City,170,Murray Hill,7.51
4,224,0.94,6.0,11.16,Alphabet City,224,Stuy Town/Peter Cooper Village,11.87


## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z2 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitabilty)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

In [0]:
####################################################################################################################
######################################## Solution implementation to task 3.1 #######################################
####################################################################################################################

def t31_summarise_trips(df):
    
    interzonal_travel = (df
    .groupBy("PULocationID", "DOLocationID")
    .agg(mean('unit_profitability').alias('average_unit_profit'),
        count('*').alias('total_trips'),
        sum('passenger_count').alias('total_passengers'))
    )
    return interzonal_travel

In [0]:
# execute task 3.1
graph = t31_summarise_trips(trips_22)

print_count(graph)
display(graph.take(10))

Row count: 38,007

PULocationID,DOLocationID,average_unit_profit,total_trips,total_passengers
87,33,6.953301339285716,4480,7930.0
90,142,7.272077669902914,14420,26941.0
90,231,7.602627789776814,19446,40818.0
234,144,9.360601338578652,19573,43232.0
116,127,4.430286355979788,1781,2695.0
129,95,5.767637600494742,1617,2461.0
142,144,5.8037569352708065,3785,6780.0
148,262,4.79148040859887,6559,11583.0
246,249,9.348631435631077,19831,42642.0
119,263,4.66326923076923,52,74.0


In [0]:
####################################################################################################################
######################################## Solution implementation to task 3.2 #######################################
####################################################################################################################

def t32_summarise_zones_pairs(df, zones_df = zone_names):
    
    df_zones = df.groupby('PULocationID').sum()\
            .join(zones_df,df.PULocationID == zones_df.LocationID, "left")

    return df_zones

# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    
    df_zones = df_zones.sort('sum(total_trips)',ascending = False)\
                        .select('Zone','sum(total_trips)').limit(10)
    return df_zones

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    
    df_zones = df_zones.sort('sum(average_unit_profit)',ascending = False)\
                                .select('Zone','sum(average_unit_profit)').limit(10)
    return df_zones

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
   
    df_zones = df_zones.sort('sum(total_passengers)',ascending = False)\
                                .select('Zone','sum(total_passengers)').limit(10)
    return df_zones

In [0]:
# execute task 3.2
zones = t32_summarise_zones_pairs(graph)

top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)

In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_trips.toPandas())

Zone,sum(total_trips)
JFK Airport,2057519
LaGuardia Airport,1954274
Midtown Center,1243545
Midtown East,1148799
Penn Station/Madison Sq West,1133300
Upper East Side South,1105008
Times Sq/Theatre District,1088403
Murray Hill,1037932
Clinton East,1036224
Upper East Side North,1030272


In [0]:
# use 'display()' return a pandas DataFrame for 'pretty' output
display(top10_profit.withColumn('sum(average_unit_profit)',round(col('sum(average_unit_profit)'),2)).toPandas())

Zone,sum(average_unit_profit)
Newark Airport,13029.7
NV,6956.62
Rockaway Park,5823.81
Midtown Center,5271.26
Arden Heights,4623.79
null,4589.01
Murray Hill,4163.6
Elmhurst,3784.11
Little Italy/NoLiTa,3498.42
Midtown South,3452.76


In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
display(top10_passenger.toPandas())

Zone,sum(total_passengers)
JFK Airport,3725934.0
LaGuardia Airport,3391754.0
Midtown Center,2716963.0
Midtown East,2477301.0
Upper East Side South,2442907.0
Penn Station/Madison Sq West,2441934.0
Times Sq/Theatre District,2352681.0
Upper East Side North,2266969.0
Clinton East,2229406.0
Murray Hill,2209266.0


## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

In [0]:
# after developing your solution, it may be convenient to combine all your functions in a single cell (at the start or end of the notebook)

# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
SIZE = 'XXL'
DATA_FORMAT = 'delta'
WITH_TASK_12 = True 

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

Trips dataset loaded!
 ---
 Size: XXL
 Format: delta
 Tables loaded: /FileStore/tables/taxi/taxi-XXL-delta/
 Number of trips (dataset rows): 132,396,785

In [0]:
# ALL FUNCTIONS IN ONE CELL TO RUN FOR TASK 4

####################################################################################################################
################################ Your solution implementation to task 1.1 goes HERE ################################
####################################################################################################################

### selecting necessary features
cols = ['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']
trips = trips.select(cols)

def t11_remove_zeros(df):
    remove_0_records = df.where('trip_distance != 0').where('passenger_count != 0')\
    .where('total_amount !=0')
 
    return remove_0_records

####################################################################################################################
################################ Your solution implementation to task 1.2 goes HERE ################################
####################################################################################################################

def t12_remove_outliers(df):
    
    MAD_distance = df.groupby('PULocationID').agg(expr('percentile(trip_distance, array(0.5))')[0].alias('median'))\
            .join(trips_11, "PULocationID", "left").withColumn("difference_median",
            abs(col('trip_distance')- col('median'))).groupby('PULocationID', 'median')\
            .agg(expr('percentile(difference_median, array(0.5))')[0].alias('median_absolute_difference'))\
            .withColumn('MAD',col('median_absolute_difference')*1.483)
    
    MAD_amount = df.groupby('PULocationID').agg(expr('percentile(total_amount, array(0.5))')[0].alias('amount_median'))\
            .join(trips_11, "PULocationID", "left").withColumn("amount_difference_median",
            abs(col('total_amount')- col('amount_median'))).groupby('PULocationID', 'amount_median')\
            .agg(expr('percentile(amount_difference_median, array(0.5))')[0].alias('amount_median_absolute_difference'))\
            .withColumn('MAD_amount',col('amount_median_absolute_difference')*1.483)

    outliersremoved_distance = df.join(MAD_distance, "PULocationID", "left")\
        .filter(abs(col("trip_distance")- col("median")) <= (col("MAD")*3.5)).select(cols)
    
    outliersremoved_amount = df.join(MAD_amount, "PULocationID", "left")\
        .filter(abs(col("total_amount")- col("amount_median")) <= (col("MAD_amount")*3.5)).select(cols)
    
    outliersremoved = outliersremoved_distance.join(outliersremoved_amount,
                                   ['PULocationID','DOLocationID','trip_distance','passenger_count','total_amount'],'inner').drop_duplicates()

    return outliersremoved

####################################################################################################################
################################ Your solution implementation to task 2.1 goes HERE ################################
####################################################################################################################

def t21_join_zones(df, zones_df = zone_names):
    # input: output of task 1.2 and zone_names dataset
    
    joined = df.join(zones_df.select('LocationID','Zone'),df.PULocationID == zones_df.LocationID,'inner')\
                .withColumnRenamed('Zone','PUzone').drop('LocationID')\
                .join(zones_df.select('LocationID','Zone'),df.DOLocationID == zones_df.LocationID,'inner')\
                .withColumnRenamed('Zone','DOzone')
    
    return joined

####################################################################################################################
################################ Your solution implementation to task 2.2 goes HERE ################################
####################################################################################################################

def t22_calc_profit(df):
    # input: output of task 2.1
    unitProf = trips_21.withColumn('unit_profitability',round(col('total_amount')/col('trip_distance'),2))
    return unitProf

####################################################################################################################
################################ Your solution implementation to task 3.1 goes HERE ################################
####################################################################################################################

def t31_summarise_trips(df):
    
    interzonal_travel = (df
    .groupBy("PULocationID", "DOLocationID")
    .agg(mean('unit_profitability').alias('average_unit_profit'),
        count('*').alias('total_trips'),
        sum('passenger_count').alias('total_passengers'))
    )
    return interzonal_travel

####################################################################################################################
################################ Your solution implementation to task 3.2 goes HERE ################################
####################################################################################################################

def t32_summarise_zones_pairs(df, zones_df = zone_names):
    
    df_zones = df.groupby('PULocationID').sum()\
            .join(zones_df,df.PULocationID == zones_df.LocationID, "left")

    return df_zones

# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    
    df_zones = df_zones.sort('sum(total_trips)',ascending = False)\
                        .select('Zone','sum(total_trips)').limit(10)
    return df_zones

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    
    df_zones = df_zones.sort('sum(average_unit_profit)',ascending = False)\
                                .select('Zone','sum(average_unit_profit)').limit(10)
    return df_zones

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
   
    df_zones = df_zones.sort('sum(total_passengers)',ascending = False)\
                                .select('Zone','sum(total_passengers)').limit(10)
    return df_zones

In [0]:
# run and record the resulting execution time shown by databricks (on the cell footer)

# IMPORTANT: this function calls all task functions in order of occurrence. For this code to run without errors, you have to load into memory all of the previous task-specific functions, even if you haven't implemented these yet.

pipeline(trips, with_task_12 = WITH_TASK_12)

Out[36]: [DataFrame[Zone: string, sum(total_trips): bigint],
 DataFrame[Zone: string, sum(average_unit_profit): double],
 DataFrame[Zone: string, sum(total_passengers): double]]

_Table 1. Pipeline performance for `parquet` format._

| metric                      | S    | M    | L    |
|-----------------------------|------|------|------|
| rows (M)                    | 2.9 | 15.6 | 42.0 |
| execution time   (w/o 1.2)  | 26.4 | 243.6 | 2823.0 |
| execution time              | 33.4 | 417.0 | 4248.0 |
| sec / 1M records (w/o 1.2)  | 9.1 | 15.5  | 67.2  |
| sec / 1M records            | 11.5 | 26.7 | 101.1  |

_Table 2. Pipeline performance for `delta` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    | 2.9 | 15.6 | 42.0 |  90.4 |  132.4 |
| execution time   (w/o 1.2)  | 23.6 | 63.0 | 163.2 | 375.0 | 574.8 |
| execution time              | 32.2 | 85.8 | 219.0 | 499.8 | 754.8 |
| sec / 1M records (w/o 1.2)  | 8.1  | 4.0  | 3.9  | 4.1  | 4.3  |
| sec / 1M records            | 11.1 | 5.5  | 5.2  | 5.5  | 5.7  |

## Comments

- `Parquet` and `delta` formats have a significant impact on the performance of python scripts running on Spark.
- On the whole, `delta` formats performed much better with lower execution times - ranging from equivalent to 100 times faster. As can be seen from the results tables above, as the dataset size increases there is a greater disparity in performance across the two formats.
- `Parquet` format applied to large datasets (sizes L, XL and XXL and from 40GB) resulted in several errors and failed tasks due to resourcing constraints (in **BOLD** font). Hence the instruction not to record timings for large sizes.
- *`Parquet`*format execution times were indirectly proportional to the dataset size, i.e. the bigger the dataset, the higher the sec / 1M record figure. However, `delta` formats ran fairly consistently across all dataset sizes.

#### Insight
For datasets greater than 5GB it is worth formatting as `delta` as this delivers a constant execution time ratio. The experience with `parquet` was that there was a proportional increase in execution time ratio (i.e. proportional lower sec / 1M performance).